In [56]:
import rclpy

rclpy.shutdown()

In [57]:
from nav2_simple_commander.robot_navigator import BasicNavigator

rclpy.init()
nav = BasicNavigator()

In [58]:
nav.waitUntilNav2Active()

[INFO] [1719992851.838724598] [basic_navigator]: Nav2 is ready for use!


In [60]:
from geometry_msgs.msg import PoseStamped

goal_pose = PoseStamped()
goal_pose.header.frame_id = 'map'
goal_pose.header.stamp = nav.get_clock().now().to_msg()
goal_pose.pose.position.x = 40.0
goal_pose.pose.position.y = 0.6
goal_pose.pose.position.z = 0.0
goal_pose.pose.orientation.x = 0.0
goal_pose.pose.orientation.y = 0.0
goal_pose.pose.orientation.z = 0.0
goal_pose.pose.orientation.w = 100.0

In [61]:
nav.goToPose(goal_pose)

[INFO] [1719992902.075162398] [basic_navigator]: Navigating to goal: 40.0 0.6...


True

In [7]:
from rclpy.duration import Duration

i = 0
while not nav.isTaskComplete():
    i = i + 1
    feedback = nav.getFeedback()
    if feedback and i % 5 == 0:
        print('Distance remaining: ' + '{:.2f}'.format(feedback.distance_remaining) + ' meters.')

        if Duration.from_msg(feedback.navigation_time) > Duration(seconds=10.0):
            nav.cancelTask()

Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.


[INFO] [1719922480.870433138] [basic_navigator]: Canceling current task.


In [8]:
from nav2_simple_commander.robot_navigator import TaskResult

result = nav.getResult()
if result == TaskResult.SUCCEEDED:
    print('Goal succeeded!')
elif result == TaskResult.CANCELED:
    print('Goal was canceled!')
elif result == TaskResult.FAILED:
    print('Goal failed!')

Goal was canceled!


In [10]:
rclpy.shutdown()

In [11]:
import rclpy
from geometry_msgs.msg import PoseWithCovarianceStamped

rclpy.init()
pose_current = PoseWithCovarianceStamped()
pose_current

geometry_msgs.msg.PoseWithCovarianceStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=0, nanosec=0), frame_id=''), pose=geometry_msgs.msg.PoseWithCovariance(pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.0, y=0.0, z=0.0), orientation=geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=0.0, w=1.0)), covariance=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])))

In [12]:
pose_current.pose.pose.position

geometry_msgs.msg.Point(x=0.0, y=0.0, z=0.0)

In [13]:
pose_current.pose.pose.orientation

geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=0.0, w=1.0)

In [14]:
pose_node = rclpy.create_node('sub_pose')

def callback(data):
    print("===")
    print(data.pose.pose.position)
    print(data.pose.pose.orientation)

In [15]:
pose_node.create_subscription(PoseWithCovarianceStamped, '/amcl_pose', callback, 10)

In [16]:
rclpy.spin_once(pose_node)

===
geometry_msgs.msg.Point(x=-0.987505236876493, y=4.917915300026402, z=0.0)
geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=-0.9854789018031215, w=0.16979792136805902)


In [17]:
pose_node = rclpy.create_node('sub_pose')

def callback(data):
    global pose_current
    print("===")
    print(data.pose.pose.position)
    print(data.pose.pose.orientation)
    pose_current = data

[WARN] [1719973893.162440999] [rcl.logging_rosout]: Publisher already registered for provided node name. If this is due to multiple nodes with the same name then all logs for that logger name will go out over the existing publisher. As soon as any node with that name is destructed it will unregister the publisher, preventing any further logs for that name from being published on the rosout topic.


In [18]:
from tf_transformations import euler_from_quaternion

pose_current

geometry_msgs.msg.PoseWithCovarianceStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=0, nanosec=0), frame_id=''), pose=geometry_msgs.msg.PoseWithCovariance(pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.0, y=0.0, z=0.0), orientation=geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=0.0, w=1.0)), covariance=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])))

In [21]:
pose_current.pose.pose.orientation

q = [0, 0, 0, 0]
q[0] = pose_current.pose.pose.orientation.x
q[1] = pose_current.pose.pose.orientation.y
q[2] = pose_current.pose.pose.orientation.z
q[3] = pose_current.pose.pose.orientation.w

euler_from_quaternion(q)

(0.0, -0.0, 0.0)

In [22]:
import numpy as np

def convert_degree(input):
    return np.array(input) * 180. / np.pi

convert_degree(euler_from_quaternion(q))

array([ 0., -0.,  0.])

In [24]:
from tf_transformations import quaternion_from_euler

to_radian = np.pi / 180.
tmp = [0, 0, 0]
print(np.array(tmp)*to_radian)

quaternion_from_euler(tmp[0], tmp[1], tmp[2])

[0. 0. 0.]


(0.0, 0.0, 0.0, 1.0)

In [29]:
from nav2_simple_commander.robot_navigator import BasicNavigator
from geometry_msgs.msg import PoseStamped
nav = BasicNavigator()

target = quaternion_from_euler(tmp[0], tmp[1], tmp[2])
goal_pose = PoseStamped()

goal_pose.header.frame_id = 'map'
goal_pose.header.stamp = nav.get_clock().now().to_msg()
goal_pose.pose.orientation.x = target[0]
goal_pose.pose.orientation.y = target[1]
goal_pose.pose.orientation.z = target[2]
goal_pose.pose.orientation.w = target[3]

In [30]:
from rclpy.duration import Duration

nav.goToPose(goal_pose)

i = 0
while not nav.isTaskComplete():
    i = i + 1
    feedback = nav.getFeedback()
    if feedback and i % 5 == 0:
        print('Distance remaining: ' + '{:.2f}'.format(feedback.distance_remaining) + ' meters.')

        if Duration.from_msg(feedback.navigation_time) > Duration(seconds=10.0):
            nav.cancelTask()

[INFO] [1719975138.944225937] [basic_navigator]: Navigating to goal: 0.0 0.0...


Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance remaining: 0.00 meters.
Distance r

[INFO] [1719975156.464427023] [basic_navigator]: Canceling current task.
